<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/07_document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 単語埋め込みを使った文書分類
* 今回も、IMDbデータセットの感情分析を文書分類問題として解く。
* ただし今回は、fastTextのような学習済みの単語埋め込みは使わない。
* 単語埋め込み自体の学習も、ネットワークの学習と同時におこなう。
* IMDbデータの準備も、`torch.torchtext`を使っておこなう。
 * つまりすべてをPyTorchのなかでおこなう。
* 参考資料
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

## データをどう扱うか
* ネットワークへの入力は、単語埋め込みを、単語の出現順どおりに並べた列にする。
 * ミニバッチは[ミニバッチのなかでの最大文書長, ミニバッチのサイズ, 単語埋め込み次元数]という形の3階のテンソルになる。
* そして、前向き計算のなかではじめて、単語埋め込みの平均をとることにする。
 * `.mean(0)`と、軸0で平均をとることになる。


## 07-00 Google Colabのランタイムのタイプを変更する
* Google ColabのランタイムのタイプをGPUに変更しておこう。
 * 上のメニューの「ランタイム」→「ランタイムのタイプを変更」→「ハードウェア　アクセラレータ」から「GPU」を選択

## 07-01 torchtextを使ってIMDbデータを読み込む
* ここでIMDbデータセットの読み込みにつかう`torchtext.datasets`については、下記を参照。
 * https://torchtext.readthedocs.io/en/latest/datasets.html

### 実験の再現性確保のための設定など
* https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, LabelField, BucketIterator

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)
torch.set_deterministic(True)

### torchtextのフィールド
* TEXTフィールドと、LABELフィールドという２種類のFieldオブジェクトのインスタンスを作る。
 * Fieldクラスの詳細については[ここ](https://github.com/pytorch/text/blob/master/torchtext/data/field.py)を参照。
* TEXTフィールドは、テキストの前処理の仕方を決めておくのに使う。
 * tokenizerは、デフォルトでは単にstring型のsplitメソッドを適用するだけになる。これは高速だが、tokenizationとしては雑。
* LABELフィールドは、ラベルの前処理に使う。

In [ ]:
TEXT = Field(tokenize="spacy")
LABEL = LabelField()

### IMDbデータセットをダウンロードした後、前処理しつつ読み込む
* ダウンロードはすぐ終わるが、解凍に少し時間がかかる。
* また、TEXTフィールドでspaCyのtokenizationを使うように設定したので、少し時間がかかる。
 * string型のsplitメソッドでtokenizeすると、時間はあまりかからない。（そのかわり、やや雑なtokenizationになる。）

In [ ]:
train_valid_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

### 最初の文書を見てみる

In [ ]:
print(train_valid_data[0].text)

['A', 'couple(Janet', 'and', 'Richard', ')', 'go', 'camping', 'out', 'in', 'the', 'woods', 'near', 'a', 'giant', 'swamp', '.', 'After', 'camping', 'and', 'enjoying', 'nature', ',', 'the', 'couple', 'takes', 'shelter', 'in', 'what', 'they', 'think', 'is', 'an', 'abandoned', 'farm', 'house', '.', 'Soon', ',', 'a', 'pair', 'of', 'escaped', 'convicts', 'show', 'up', 'and', ',', 'after', 'much', 'delaying', 'of', 'the', 'inevitable', ',', 'they', 'proceed', 'to', 'rape', 'Janet', 'and', 'lock', 'Richard', 'in', 'a', 'birdcage.<br', '/><br', '/>This', 'LAST', 'HOUSE', 'ON', 'THE', 'LEFT', '-', 'like', 'film', 'has', 'to', 'be', 'one', 'of', 'the', 'most', 'underrated', 'horror', 'films', 'ever', 'made', '.', 'It', "'s", 'one', 'of', 'the', 'more', 'sick', 'and', 'twisted', 'early', '70s', 'shockers', '.', 'Moreover', ',', 'I', 'found', 'this', 'to', 'be', 'quite', 'enchanting', 'and', 'beautiful', 'in', 'it', "'s", 'perverse', 'tone', '.', 'I', 'love', 'CAGED', 'TERROR', '.', 'The', 'music',

In [ ]:
print(train_valid_data[0].label)

pos


### テストセット以外の部分を訓練データと検証データに分ける

In [ ]:
train_data, valid_data = train_valid_data.split(split_ratio=0.8)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [ ]:
print(train_data[0].text)

['I', 'have', 'complained', 'to', 'ABC', 'about', 'the', 'cancellation', 'of', 'six', 'degrees', '.', 'If', 'enough', 'people', 'do', 'the', 'same', 'then', 'it', 'could', 'be', 'enough', 'to', 'bring', 'this', 'fabulous', 'show', 'back', 'to', 'life', '!', '!', 'Just', 'go', 'onto', 'the', 'official', 'site', 'and', 'the', 'rest', 'is', 'simple', 'enough', '.', 'I', 'do', 'not', 'understand', 'why', 'this', 'show', 'has', 'been', 'cancelled', '.', 'What', 'a', 'fantastic', 'show', ',', 'cast', 'and', 'characters', '.', 'The', 'whole', 'concept', 'is', 'gripping', 'viewing', '!', 'I', 'am', 'astounded', 'that', 'my', 'favourite', 'show', 'is', 'over', 'after', 'just', 'one', 'series', '.', 'Why', 'is', 'this', '?', 'Six', 'degrees', 'is', 'phenomenal', ',', 'it', "'s", 'better', 'than', 'so', 'many', 'other', 'TV', 'programmes', 'out', 'there', '!', 'Until', 'I', 'heard', 'they', 'were', 'stopping', 'it', 'from', 'a', 'friend', 'it', 'had', "n't", 'even', 'occurred', 'to', 'me', 'that'

### データセットの語彙とラベルを作る
* TEXTラベルのほうでは、最大語彙サイズを指定する。

In [ ]:
MAX_VOCAB_SIZE = 25000 # この値は適当。

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

なぜ語彙サイズが25,000ではなく25,002なのかについては、少し下の説明を参照。

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


### 出現頻度順で上位２０単語を見てみる

In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 231080), (',', 219679), ('.', 189780), ('and', 125114), ('a', 124787), ('of', 114842), ('to', 106981), ('is', 87533), ('in', 70047), ('I', 62069), ('it', 61265), ('that', 56156), ('"', 50707), ("'s", 49452), ('this', 48451), ('-', 42013), ('/><br', 40450), ('was', 39644), ('as', 34848), ('with', 34297)]


### 単語ID順に最初の１０単語を見てみる
* IDのうち、0と1は、未知語とパディング用の単語という特殊な単語に割り振られている。
 * 未知語は`<unk>`という特殊な単語に置き換えられる。これのIDが0。
 * パディングとは、長さが不揃いの複数の文書を同じミニバッチにまとめるとき、すべての文書の長さを無理やりそろえるため、文書末尾に特殊な単語（元々の語彙にない、人工的に用意した単語）を追加すること。
 * パディング用の単語が`<pad>`になっているのは、上のほうで使ったFieldクラスのインスタンスを作るときのデフォルトの値がこの`<pad>`になっているため。

In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


### ラベルのほうのIDを確認する
* こちらはnegとposに対応する２つのIDしかない。

In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f277c7d11e0>, {'neg': 0, 'pos': 1})


### ミニバッチを取り出すためのiteratorを作る
* ミニバッチのサイズを指定する。
 * ミニバッチのサイズは、性能を出すためにチューニングすべきハイパーパラメータのひとつ。

In [ ]:
BATCH_SIZE = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator = data.BucketIterator(train_data, batch_size=BATCH_SIZE, device=device,
                                     sort_within_batch=True, shuffle=True, sort_key=lambda x: len(x.text))
valid_iterator = data.BucketIterator(valid_data, batch_size=BATCH_SIZE, device=device)
test_iterator = data.BucketIterator(test_data, batch_size=BATCH_SIZE, device=device)

### ミニバッチの中身を確認する

* 訓練データのiteratorを回してミニバッチをすべて取得してみる
 * ミニバッチのshapeを表示してみる

In [ ]:
for i, batch in enumerate(train_iterator):
  print(i, batch.text.shape)

0 torch.Size([133, 100])
1 torch.Size([458, 100])
2 torch.Size([247, 100])
3 torch.Size([212, 100])
4 torch.Size([105, 100])
5 torch.Size([380, 100])
6 torch.Size([94, 100])
7 torch.Size([185, 100])
8 torch.Size([285, 100])
9 torch.Size([112, 100])
10 torch.Size([50, 100])
11 torch.Size([538, 100])
12 torch.Size([188, 100])
13 torch.Size([161, 100])
14 torch.Size([226, 100])
15 torch.Size([135, 100])
16 torch.Size([391, 100])
17 torch.Size([835, 100])
18 torch.Size([180, 100])
19 torch.Size([169, 100])
20 torch.Size([242, 100])
21 torch.Size([929, 100])
22 torch.Size([1064, 100])
23 torch.Size([69, 100])
24 torch.Size([298, 100])
25 torch.Size([268, 100])
26 torch.Size([329, 100])
27 torch.Size([215, 100])
28 torch.Size([622, 100])
29 torch.Size([128, 100])
30 torch.Size([274, 100])
31 torch.Size([223, 100])
32 torch.Size([154, 100])
33 torch.Size([475, 100])
34 torch.Size([63, 100])
35 torch.Size([173, 100])
36 torch.Size([404, 100])
37 torch.Size([440, 100])
38 torch.Size([121, 100])

* ミニバッチの形は、[ミニバッチ内での最大文書長, ミニバッチのサイズ]になっていることに注意！
 * ミニバッチのサイズが最初に来ているのではない！
 * [ミニバッチのサイズ, ミニバッチ内での最大文書長]という形にしたいなら、テキストのfieldを作るとき以下のようにする。

__`TEXT = data.Field(tokenize="spacy", batch_first=True)`__

* 上記のループを抜けたあとには、変数batchには最後のミニバッチが代入されている。

In [ ]:
batch.text.shape

torch.Size([139, 100])

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を表示させてみる。

In [ ]:
print(batch.text[:, 0])

tensor([   11,    97,   133,   995,     2,   406,    11,   236,    16,    22,
            4,   357,    79,    29,     6, 10165,    29,     0,    10,     2,
         2902,  8669,     5,    16,    19,     2, 19088,    22,     7,     2,
         2498,     4,   357,  2384,   542,    64,   139,     2,   375,  5891,
            4,   239,    86,    33,    80,   378,     8,   140,    12,    41,
            2,  2080,    17,   406,   820,     4,     0,    23,     6, 18057,
          425,  3810,    66,     9,    42,    11,    74,    34,     8,   153,
           26,    45,    34,    16,   407,  3129,    13,    83,   988,   228,
         3404,   872,   444,     4,    11,   171,    33,   433,     8,  6890,
            2,   988,    21,  1546,   759,    48,    11,    97,    55,  1753,
            8, 16358,   382,    11,    93,    83,   872,   444,     7, 19338,
            4,    11,    91,    33,   180,   755,     0,  3643,    10,     2,
          942,    26,    11,   426,    99,    10,     2,    22, 

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を単語列に戻したものを表示させてみる。

In [ ]:
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:, 0]]))

I will never forget the night I saw this movie . We were on a submarine on <unk> in the North Atlantic and this was the scheduled movie of the evening . We ALL gave up after the second reel . They did not even try to show it at the mid - night showing . <unk> for a rerun instead ...... This is all I really have to say but they have this stupid rule that my comment must contain ten lines . I 'm not supposed to pad the comment with random words so I will just continue to ramble until I get my ten lines of BS . I could not find George <unk> listed in the credits but I remember him in the movie . The <unk> was terrible and the songs even worse .


* このミニバッチに含まれる文書のうち、最後の文書の単語ID列を表示させてみる。

In [ ]:
print(batch.text[:, BATCH_SIZE-1])

tensor([12995,     3,  7102,  4776,   221,    22,  3786,     4,    11,    73,
          355,  1887,    23,    16,    22,     3,     5,     2,   548,     7,
         5743,  2604,  1057,     0,     9,    38,    13,   156,    57,    28,
         8947,     3,    26,    62,  1901,    57,   156,    78,     6,   784,
           20,  3394,     5,  3024,    20,    16,    38,    19,     4,    66,
           63,    34,    92, 10355,     3,    20,    12,   230,     9,    10,
          168,  2641,     7,   112,     3,    70,     2,   221,     0,     3,
         1112,     3,    12,    86,    33,     4,    25,   221,    84,  1718,
            2,  5902,     0,     5,   134,     2,   374,   907,     7,  5743,
          221,     3,    26,    12,    15,    33,    62,  1233,     3, 23158,
         2226,     3,     5,    10,   213,   448,   360,  3434,    27,   570,
           16,     9,   732,     8,   863,    17,   839,    23,  8550,   825,
            3,    12,    84,   140,  4396,    10,   132,  1410, 

最後の文書の末尾は「1」で埋められていることが分かる。

この1は、パディング用単語のIDだったことを想起されたい。

ミニバッチに含まれる文書の長さを調べると、文書が文書長の降順に並べられていることが分かる。

In [ ]:
(batch.text != 1).sum(0)

tensor([139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139,
        139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139,
        139, 139, 139, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 137, 137, 137, 137, 137, 137, 137, 137,
        137, 137], device='cuda:0')

## 07-02 MLPによる文書分類の準備
* 今回は、ごく簡単なMLPで文書分類をする。
* 文書中の全単語トークンの埋め込みベクトルの平均を、MLPの入力とする。
 * 当然、語順の情報は使われない。
 * つまり、bag-of-wordsモデルになっている。

### 定数の設定
* 単語埋め込みベクトルの次元数は128にする。

In [ ]:
INPUT_DIM = len(TEXT.vocab)
NUM_CLASS = len(LABEL.vocab)
EMBED_DIM = 128
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

print(f'語彙サイズ {INPUT_DIM}, クラス数 {NUM_CLASS}, 単語埋め込み次元 {EMBED_DIM}')

語彙サイズ 25002, クラス数 2, 単語埋め込み次元 128


### モデルを定義する前にPyTorchの単語埋め込みがどんなものかを見てみる

以下のように、語彙サイズと埋め込みの次元数を指定しつつ、torch.nn.Embeddingのインスタンスを作ればよい。

In [ ]:
embed = nn.Embedding(INPUT_DIM, EMBED_DIM, padding_idx=PAD_IDX)

パディング用の単語の埋め込みはゼロベクトルになる。

In [ ]:
print(embed(torch.tensor([21,1])))

tensor([[ 1.4663,  0.3888,  0.5244, -0.6720,  1.2702,  0.1475, -0.6756,  1.0647,
          0.4122,  0.4360,  0.4711,  1.7838, -0.2890, -0.1943, -0.4321,  0.1552,
         -0.2901,  0.6956,  0.3870,  1.3461,  3.0132,  0.9346, -2.4039, -0.0521,
         -0.7123, -0.9902,  1.6379,  0.2446, -0.4733,  1.0840, -0.1715, -1.9957,
         -0.7140,  0.2289, -0.5881,  2.2581, -0.5802,  1.4646,  0.0392, -0.7295,
          0.4982, -1.1848,  0.4286, -0.0767,  1.2760, -1.3223,  0.0627,  0.3617,
         -0.7071,  0.0949, -0.5615, -0.6247,  1.0816,  0.7235, -0.9131,  0.6759,
          0.0999, -0.9366,  1.9773,  1.2333,  0.1287, -1.0535, -1.6376,  0.7970,
          0.1361, -0.0714, -0.2324, -0.8216,  1.4716, -0.0678,  0.4309, -0.1257,
          0.2348, -1.3550,  0.3339,  2.1095,  1.8534, -0.8028,  0.6104,  0.8844,
         -0.4362,  0.4587,  0.4177,  1.4428,  0.0063, -0.2002,  0.4557,  0.4221,
         -1.5692, -0.5961, -0.7103, -0.9230,  0.1227, -0.0557, -0.0169,  0.9816,
          0.1775,  0.8781, -

### モデルの定義
* MLP（多層パーセプトロン）だが、入り口に単語埋め込み層が挿入されている。

In [ ]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.embed(text)
    x = x.mean(0) # 文書に含まれる全単語トークンの単語ベクトルの平均
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

### モデルを作る
* モデル（のインスタンス）をGPUに移動させている点に注意。

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX).to(device)

### 損失関数とoptimizerとschedulerを作る

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

### 訓練用の関数
* 最初の`model.train()`に注意。こうやって、モデルを訓練モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [ ]:
def train(data_iterator, model, optimizer, scheduler, criterion):

  model.train()

  train_loss = 0
  train_acc = 0
  for batch in data_iterator:
    optimizer.zero_grad()
    text, cls = batch.text, batch.label
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).float().mean().item()

  scheduler.step()

  num_batch = len(data_iterator)
  return train_loss / num_batch, train_acc / num_batch

### 評価用の関数
* 最初の`model.eval()`に注意。こうやって、モデルを評価モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [ ]:
def test(data_iterator, model, criterion):

  model.eval()

  loss = 0
  acc = 0
  for batch in data_iterator:
    text, cls = batch.text, batch.label
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      loss += loss.item()
      acc += (output.argmax(1) == cls).float().mean().item()

  num_batch = len(data_iterator)
  return loss / num_batch, acc / num_batch

## 07-03 分類器の訓練と検証セットでの評価

In [ ]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 1 | time in 0 minutes, 2 seconds | lr=0.009500
	Loss: 0.38365(train)	|	Acc: 82.22%(train)
	Loss: 0.01790(valid)	|	Acc: 88.22%(valid)
Epoch 2 | time in 0 minutes, 2 seconds | lr=0.009025
	Loss: 0.16225(train)	|	Acc: 93.99%(train)
	Loss: 0.01481(valid)	|	Acc: 88.52%(valid)
Epoch 3 | time in 0 minutes, 2 seconds | lr=0.008574
	Loss: 0.07868(train)	|	Acc: 97.22%(train)
	Loss: 0.01262(valid)	|	Acc: 86.78%(valid)
Epoch 4 | time in 0 minutes, 2 seconds | lr=0.008145
	Loss: 0.03622(train)	|	Acc: 98.72%(train)
	Loss: 0.01336(valid)	|	Acc: 87.76%(valid)
Epoch 5 | time in 0 minutes, 2 seconds | lr=0.007738
	Loss: 0.02071(train)	|	Acc: 99.30%(train)
	Loss: 0.00742(valid)	|	Acc: 87.70%(valid)
Epoch 6 | time in 0 minutes, 2 seconds | lr=0.007351
	Loss: 0.01853(train)	|	Acc: 99.40%(train)
	Loss: 0.01542(valid)	|	Acc: 87.30%(valid)
Epoch 7 | time in 0 minutes, 2 seconds | lr=0.006983
	Loss: 0.01147(train)	|	Acc: 99.58%(train)
	Loss: 0.01617(valid)	|	Acc: 87.88%(valid)
Epoch 8 | time in 0 minutes

## 07-04 再検討
* 訓練データ上での分類精度がほぼ100%になってしまっている。
* 検証データでの分類精度と大きな差があり、明らかにオーバーフィッティング。

### ドロップアウトを使う
* モデルのインスタンスを作るときにdropoutの確率を引数pで指定できるようにする。

In [ ]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx, p=0.0):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.dropout = nn.Dropout(p=p)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.dropout(self.embed(text)) #埋め込み層の直後にdropout
    x = x.mean(0)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 1 | time in 0 minutes, 2 seconds | lr=0.009500
	Loss: 0.43358(train)	|	Acc: 78.99%(train)
	Loss: 0.01906(valid)	|	Acc: 87.42%(valid)
Epoch 2 | time in 0 minutes, 2 seconds | lr=0.009025
	Loss: 0.22468(train)	|	Acc: 91.23%(train)
	Loss: 0.01557(valid)	|	Acc: 88.08%(valid)
Epoch 3 | time in 0 minutes, 2 seconds | lr=0.008574
	Loss: 0.15242(train)	|	Acc: 94.25%(train)
	Loss: 0.01521(valid)	|	Acc: 88.64%(valid)
Epoch 4 | time in 0 minutes, 2 seconds | lr=0.008145
	Loss: 0.10738(train)	|	Acc: 96.01%(train)
	Loss: 0.01225(valid)	|	Acc: 88.48%(valid)
Epoch 5 | time in 0 minutes, 2 seconds | lr=0.007738
	Loss: 0.07579(train)	|	Acc: 97.20%(train)
	Loss: 0.01259(valid)	|	Acc: 88.74%(valid)
Epoch 6 | time in 0 minutes, 2 seconds | lr=0.007351
	Loss: 0.05858(train)	|	Acc: 97.82%(train)
	Loss: 0.01455(valid)	|	Acc: 87.94%(valid)
Epoch 7 | time in 0 minutes, 2 seconds | lr=0.006983
	Loss: 0.04387(train)	|	Acc: 98.33%(train)
	Loss: 0.01025(valid)	|	Acc: 88.82%(valid)
Epoch 8 | time in 0 minutes

### L２正則化を使う
* optimizerのweight_decayパラメータを0より大きな値にする。

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 1 | time in 0 minutes, 2 seconds | lr=0.009500
	Loss: 0.58818(train)	|	Acc: 68.09%(train)
	Loss: 0.02606(valid)	|	Acc: 54.48%(valid)
Epoch 2 | time in 0 minutes, 2 seconds | lr=0.009025
	Loss: 0.45343(train)	|	Acc: 79.09%(train)
	Loss: 0.02441(valid)	|	Acc: 81.82%(valid)
Epoch 3 | time in 0 minutes, 2 seconds | lr=0.008574
	Loss: 0.40164(train)	|	Acc: 82.25%(train)
	Loss: 0.02321(valid)	|	Acc: 74.08%(valid)
Epoch 4 | time in 0 minutes, 2 seconds | lr=0.008145
	Loss: 0.37071(train)	|	Acc: 83.83%(train)
	Loss: 0.02561(valid)	|	Acc: 60.28%(valid)
Epoch 5 | time in 0 minutes, 2 seconds | lr=0.007738
	Loss: 0.34453(train)	|	Acc: 85.48%(train)
	Loss: 0.02345(valid)	|	Acc: 77.32%(valid)
Epoch 6 | time in 0 minutes, 2 seconds | lr=0.007351
	Loss: 0.33281(train)	|	Acc: 85.71%(train)
	Loss: 0.02430(valid)	|	Acc: 69.64%(valid)
Epoch 7 | time in 0 minutes, 2 seconds | lr=0.006983
	Loss: 0.33043(train)	|	Acc: 86.00%(train)
	Loss: 0.02125(valid)	|	Acc: 86.98%(valid)
Epoch 8 | time in 0 minutes

### early stopping
* dev setでのaccuracyが4回連続で最高値を下回ったら訓練を終えることにする。
* early stoppingの実現については、PyTorch Lightningを使う手もある。
 * https://pytorch-lightning.readthedocs.io/en/latest/early_stopping.html

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
patience = 4
early_stop_count = 0
best_valid_acc = 0.0

MIN_N_EPOCHS = 10 # 最低このエポック数は実行する
N_EPOCHS = 50 # エポック数を増やしておく
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

  # early stopping
  if epoch + 1 > MIN_N_EPOCHS:
    if best_valid_acc <= valid_acc:
      best_valid_acc = valid_acc
      early_stop_count = 0
    else:
      early_stop_count += 1
      if early_stop_count == patience:
        break

Epoch 1 | time in 0 minutes, 2 seconds | lr=0.009500
	Loss: 0.59871(train)	|	Acc: 66.79%(train)
	Loss: 0.02251(valid)	|	Acc: 56.18%(valid)
Epoch 2 | time in 0 minutes, 2 seconds | lr=0.009025
	Loss: 0.43493(train)	|	Acc: 80.24%(train)
	Loss: 0.02186(valid)	|	Acc: 77.00%(valid)
Epoch 3 | time in 0 minutes, 2 seconds | lr=0.008574
	Loss: 0.38540(train)	|	Acc: 83.23%(train)
	Loss: 0.02093(valid)	|	Acc: 83.76%(valid)
Epoch 4 | time in 0 minutes, 2 seconds | lr=0.008145
	Loss: 0.35846(train)	|	Acc: 84.48%(train)
	Loss: 0.02126(valid)	|	Acc: 85.30%(valid)
Epoch 5 | time in 0 minutes, 2 seconds | lr=0.007738
	Loss: 0.34493(train)	|	Acc: 85.37%(train)
	Loss: 0.02423(valid)	|	Acc: 75.62%(valid)
Epoch 6 | time in 0 minutes, 2 seconds | lr=0.007351
	Loss: 0.33198(train)	|	Acc: 85.92%(train)
	Loss: 0.02041(valid)	|	Acc: 82.98%(valid)
Epoch 7 | time in 0 minutes, 2 seconds | lr=0.006983
	Loss: 0.31971(train)	|	Acc: 86.85%(train)
	Loss: 0.02184(valid)	|	Acc: 86.56%(valid)
Epoch 8 | time in 0 minutes

## 07-05 テストセット上で評価
* 見つけ出したベストな設定を使って、テストセット上での最終的な評価をおこなう。

In [ ]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_iterator, model, criterion)
print(f'\tLoss: {test_loss:.5f}(test)\t|\tAcc: {test_acc * 100:.2f}%(test)')

Checking the results of test dataset...
	Loss: 0.00390(test)	|	Acc: 86.76%(test)
